In [12]:

import numpy as np
import ProjGradAscent
import mpnet_model
import End_to_End_model
import utils_function
#import plots
import time
from pathlib import Path
import torch



In [14]:
N     = 4        # Num of users
L     = 10       # RF chains
T     = 1        # Instant
M     = 64       # BS antennas
m     = T*L      # mesures
N0    = 1e-3     # noise variance
test_size  = 1000# NUMBER OF MIMO CHANNELS  


In [15]:

#Load data
path_init=Path.cwd()
file_name = 'Data/1e-3/data_var_snr/test_data.npz'  
data = np.load(path_init/file_name)


In [16]:
h=torch.tensor(data['h'],dtype=torch.complex128)

In [17]:

H_test=(h).view(-1,N,M)
P=np.linalg.norm(H_test,ord='fro',axis=(1,2))**2 / (M*N0)
Pi=np.linalg.norm(h,2,axis=1)**2/ (M * N0)
SNR_dB= 10*np.log10(P)

In [ ]:
S1=[]
S2=[]

for i in range(H_test.shape[0]):

    # SVD of the channels
    u, lambda_, vh= torch.linalg.svd(H_test, full_matrices=True)
    g=np.abs(lambda_**2) # channel's gains^2
    g=g[i].unsqueeze(1).numpy()
    P=1  # if s2 formula will be used 
   #P=P # if s1 formula will be used


    # Bisection search for alpha 
    alpha_low = min(N0/g) # Initial low
    alpha_high = (P + np.sum(N0/g))/N # Initial high

    #print(alpha_high)
    #print(alpha_low)


    stop_threshold = 1e-15# Stop threshold


    # Iterate while low/high bounds are further than stop_threshold
    while(np.abs(alpha_low - alpha_high) > stop_threshold):

        alpha = (alpha_low + alpha_high) / 2 # Test value in the middle of low/high


        # Solve the power allocation
        p = alpha - N0/g 
     
    
        p[p < 0] = 0 # Consider only positive power allocation
        

     
        if (np.sum(p) > P): # Exceeds power limit => lower the upper bound
            #print('upper')
            alpha_high = alpha
        else: # Less than power limit => increase the lower bound

            #print('low')
            alpha_low = alpha


    # Precoder Normalization check 
    F_BB=vh[i,:,:N].squeeze()
    
    F=F_BB @ np.diag(p.squeeze())

    print(torch.linalg.norm(F,ord='fro')**2)


    #direct sum rate formula 
    h=H_test[i]

    Fopt=F
    a1 = torch.transpose(Fopt, 0, 1).conj() @ torch.transpose(h, 0, 1).conj()
    a3 = h @ Fopt @ a1
    a4 = torch.eye(N).reshape((N, N)) + a3/(N*N0)  

    
    #s1 = torch.abs(torch.log(a4.det()))

    #WF formula 
    s2= np.sum(np.log(1 + g*p/N0))



    #S1.append(s1)
    S2.append(s2)

    

In [ ]:

#print(np.mean(S1))

In [ ]:

print(np.mean(S2))

In [22]:
# save sum rate
sum_rate={}
sum_rate['wf']=np.mean(S2)
np.savez(path_init / 'sumRate'/'1e-3'/'wf.npz', **sum_rate)
